<a href="https://colab.research.google.com/github/aashitarakesh/Twitter_Analysis/blob/main/token_hash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
from google.colab import files
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [61.8 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:13 https://developer.download.nvi

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("alpha_to_numeric").getOrCreate()

In [ ]:
# # loads data from google drive

# from google.colab import drive
# drive.mount("/content/gdrive")

# pd.read_csv('/content/gdrive/My Drive/UPenn Data Analytics/UPenn Data/')

In [ ]:
# # import data from local system

# from google.colab import files
# uploaded = files.upload()
# pd.read_csv("")

In [ ]:
# # import file from github

# from pyspark import SparkFiles

# url = 'https://raw.githubusercontent.com/aashitarakesh/Twitter_Analysis/main/sentiment_twitter_clean.csv'
# spark.sparkContext.addFile(url)
# df = spark.read.csv(SparkFiles.get("sentiment_twitter_clean.csv"), sep=",", header=True)

# # Show DataFrame
# df.show()


In [3]:
# import file from github

from pyspark import SparkFiles

url = 'https://raw.githubusercontent.com/aashitarakesh/Twitter_Analysis/main/sentiment_twitter_clean.csv'
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("sentiment_twitter_clean.csv"), sep=",", header=True)

# Show DataFrame
df.show()


+-----+--------------------+-----+
|Index|              Tweets|Label|
+-----+--------------------+-----+
|  106|real good moment ...|    0|
|  217|       reading manga|    0|
|  288|need send em acco...|    0|
|  540|add myspace myspa...|    0|
|  624|sleepy good times...|    0|
|  701|nbn someone alrea...|    0|
|  808|23 24ï½c possible...|    0|
| 1193|nite twitterville...|    0|
| 1324|night darlin swee...|    0|
| 1332|good morning ever...|    0|
| 1368|finally created w...|    0|
| 1578|kisha cnt get u t...|    0|
| 1595|yes remember band...|    0|
| 1861|really love refle...|    0|
| 1889|ooo fantasy like ...|    0|
| 1899|probs sell nothin...|    0|
| 1919|quotnokla connect...|    0|
| 1992|stayed late start...|    0|
| 2097|read new job cong...|    0|
| 2126|havent able sleep...|    0|
+-----+--------------------+-----+
only showing top 20 rows



In [15]:
# # Drop all rows with missing information
# df = df.dropna(how='any')


In [4]:
# # Verify dropped rows
# df.count()
df

DataFrame[Index: string, Tweets: string, Label: string]

In [5]:
from pyspark.sql.functions import length
# Create a length column to be used as a future feature 
data_df = df.withColumn('length', length(df['Tweets']))
data_df.show()

+-----+--------------------+-----+------+
|Index|              Tweets|Label|length|
+-----+--------------------+-----+------+
|  106|real good moment ...|    0|    34|
|  217|       reading manga|    0|    13|
|  288|need send em acco...|    0|    92|
|  540|add myspace myspa...|    0|    33|
|  624|sleepy good times...|    0|    32|
|  701|nbn someone alrea...|    0|    54|
|  808|23 24ï½c possible...|    0|    28|
| 1193|nite twitterville...|    0|    30|
| 1324|night darlin swee...|    0|    25|
| 1332|good morning ever...|    0|    22|
| 1368|finally created w...|    0|    75|
| 1578|kisha cnt get u t...|    0|    45|
| 1595|yes remember band...|    0|    38|
| 1861|really love refle...|    0|    31|
| 1889|ooo fantasy like ...|    0|    37|
| 1899|probs sell nothin...|    0|    50|
| 1919|quotnokla connect...|    0|    31|
| 1992|stayed late start...|    0|    43|
| 2097|read new job cong...|    0|    44|
| 2126|havent able sleep...|    0|    51|
+-----+--------------------+-----+

In [6]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
# Create all the features to the data set
pos_neg_to_num = StringIndexer(inputCol='Label',outputCol='label')
tokenizer = Tokenizer(inputCol="Tweets", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="stop_tokens", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')

In [7]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [8]:
# Create a and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[pos_neg_to_num, tokenizer, stopremove, hashingTF, idf, clean_up])

In [9]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(data_df)
cleaned = cleaner.transform(data_df)


In [ ]:
cleaned

DataFrame[Index: string, Tweets: string, Label: string, tokenized_tweets: string, length: int, new_label: double, token_text: array<string>, stop_tokens: array<string>, hash_token: vector, idf_token: vector, features: vector]

In [10]:
# Show label and resulting features
cleaned.select(['label', 'features']).show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(262145,[76764,11...|
|  0.0|(262145,[41314,12...|
|  0.0|(262145,[942,2032...|
|  0.0|(262145,[48063,98...|
|  0.0|(262145,[73853,11...|
|  0.0|(262145,[2306,178...|
|  0.0|(262145,[22346,44...|
|  0.0|(262145,[62166,93...|
|  0.0|(262145,[88594,15...|
|  0.0|(262145,[113432,1...|
|  0.0|(262145,[2306,762...|
|  0.0|(262145,[18176,51...|
|  0.0|(262145,[14118,42...|
|  0.0|(262145,[186480,2...|
|  0.0|(262145,[13823,12...|
|  0.0|(262145,[14118,16...|
|  0.0|(262145,[93523,10...|
|  0.0|(262145,[16004,68...|
|  0.0|(262145,[10345,53...|
|  0.0|(262145,[9657,506...|
+-----+--------------------+
only showing top 20 rows



In [16]:
final_df = cleaned.select(['label', 'features']).show()
final_df

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(262145,[76764,11...|
|  0.0|(262145,[41314,12...|
|  0.0|(262145,[942,2032...|
|  0.0|(262145,[48063,98...|
|  0.0|(262145,[73853,11...|
|  0.0|(262145,[2306,178...|
|  0.0|(262145,[22346,44...|
|  0.0|(262145,[62166,93...|
|  0.0|(262145,[88594,15...|
|  0.0|(262145,[113432,1...|
|  0.0|(262145,[2306,762...|
|  0.0|(262145,[18176,51...|
|  0.0|(262145,[14118,42...|
|  0.0|(262145,[186480,2...|
|  0.0|(262145,[13823,12...|
|  0.0|(262145,[14118,16...|
|  0.0|(262145,[93523,10...|
|  0.0|(262145,[16004,68...|
|  0.0|(262145,[10345,53...|
|  0.0|(262145,[9657,506...|
+-----+--------------------+
only showing top 20 rows



In [19]:
# final_df.toPandas().to_csv('test.csv')
# final_df.write.csv('test.csv')

final_df.coalesce(1).write.format("https://raw.githubusercontent.com/aashitarakesh/Twitter_Analysis/main").option("header", "true").save("test.csv")

AttributeError: ignored

FileNotFoundError: ignored